In [33]:
import os
import pickle
import models

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import seaborn as sns
from numpyro.diagnostics import hpdi

from hbmep.nn import functional as F
from hbmep.model.utils import Site as site

from hbmep.config import Config
from hbmep.model.utils import Site as site
from scipy import stats

from models import NonHierarchicalBayesianModel

USER = os.environ["USER"]


In [2]:
src = f"/home/andres/repos/rat-mapping-paper/reports/C_SMA_LAR/non_hierarchical_bayesian_model/inference.pkl"
with open(src, "rb") as f:
    (
        df,
        encoder_dict,
        model,
        posterior_samples,
    ) = pickle.load(f)


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [3]:
a = posterior_samples[site.a]
b = posterior_samples[site.b]
L = posterior_samples[site.L]
ell = posterior_samples[site.ell]
H = posterior_samples[site.H]

x = np.linspace(0, 500, 1000)

named_params = [site.a, site.b, site.L, site.ell, site.H]
params = [posterior_samples[param][:400, ...] for param in named_params]

for named_param, param in zip(named_params, params):
    
    print(named_param, param.shape)

a (400, 8, 70, 3, 6)
b (400, 8, 70, 3, 6)
L (400, 8, 70, 3, 6)
ℓ (400, 8, 70, 3, 6)
H (400, 8, 70, 3, 6)


In [4]:
subjects = df['participant'].unique()
subjects = sorted(encoder_dict['participant'].inverse_transform(subjects))


In [15]:
norm_y = []
norm_x = []

for subject_ind, subject in enumerate(subjects):
    curr_params = [
        param[:, subject_ind, ...][..., None] for param in params
    ]
    constant = np.nanmedian(np.nanmean(curr_params[0], axis=0))

    x_temp = np.linspace(0., 5 * constant, 500)
    x_temp = x_temp[None, None, None, None, :]

    temp_thresh = F.rectified_logistic(
        x_temp,
        *curr_params
    )
    temp_thresh = temp_thresh - curr_params[2]
    norm_y.append(temp_thresh)
    norm_x.append(x_temp)

norm_y = np.array(norm_y)
norm_x = np.array(norm_x)

# print(norm_y.shape)
# print(norm_x.shape)


/tmp/ipykernel_343710/809937145.py:8: RuntimeWarning: Mean of empty slice
  constant = np.nanmedian(np.nanmean(curr_params[0], axis=0))


In [16]:
norm_y.shape

(8, 400, 70, 3, 6, 500)

In [17]:
np.isnan(norm_y[:, 0, ..., 0]).sum()

7662

In [43]:
np.isnan(norm_y).sum() / 400 / 500

10080.0

In [42]:
8*70*3*6

10080

In [21]:
compound_position = encoder_dict[model.features[1]].inverse_transform(sorted(df['compound_position'].unique()))
mono = [c for c in compound_position if c.split("-")[0] == "" or c.split("-")[1] == ""]

In [18]:
laterality = ['LL','L', 'LM','LM1', 'LM2', 'M']
segments = ["-C5", "-C6", "-C7"]

In [19]:
norm_y.shape

(8, 400, 70, 3, 6, 500)

In [22]:
arr = []
mask_biceps = [] 

# for subject in subjects:
#     for size in sizes:  
for segment in segments:
    for lat in laterality:
        cpos = f"{segment}{lat}"
        if cpos in mono:
            arr.append(norm_y[:,:,mono.index(cpos),...])

        else:
            arr.append(np.full(norm_y[:, :, 0, ...].shape, np.nan))

arr = np.array(arr)
arr = arr.reshape(len(segments), len(laterality), *arr.shape[1:])
arr.shape




(3, 6, 8, 400, 3, 6, 500)

In [23]:
arr = arr.swapaxes(0,3)
arr = arr.swapaxes(1,2)

(400, 8, 6, 3, 3, 6, 500)

In [24]:

arr.shape #400, subjects, laterality, segments, sizes, muscles, 500

(400, 8, 6, 3, 3, 6, 500)

In [27]:
big_y = arr[:,:,:,:,0,...]
big_x = arr[:,:,:,:,0,...]

In [28]:
big_y.shape

(400, 8, 6, 3, 6, 500)

In [61]:
arr = ma.masked_invalid(arr)

In [62]:
def get_entropy(size_ind):
    y = arr[...,size_ind,:,:]
    x = norm_x

    y_max = ma.max(y, axis=(2,3, -1), keepdims=True) 
    y_max.shape

    y = ma.where(y, y / y_max, 0.)
    y.shape

    p = ma.sum(y, axis=-2, keepdims=True)
    p = ma.where(y, y / p, 0.)
    p.shape

    plogp = ma.where(p, p * ma.log(p), 0.)
    plogp.shape

    entropy = ma.where(
        ma.any(p, axis=-2, keepdims=True),
        (
            1
            + (ma.sum(plogp, axis=-2, keepdims=True) / np.log(plogp.shape[-2]))
        ),
        0.
    )
    entropy = entropy[..., 0, :]
    entropy.shape

    return entropy, x

In [63]:
func_entrop, x_val = get_entropy(0)

In [64]:
func_entrop

masked_array(
  data=[[[[[0.0, 0.0, 0.0, ..., 0.10349010849973628,
            0.10349012025884963, 0.1034901293005751],
           [0.0, 0.0, 0.0, ..., 0.1584736839467874,
            0.15841767453528866, 0.15836373726350106],
           [--, --, --, ..., --, --, --]],

          [[0.0, 0.0, 0.0, ..., 0.3145863169887513,
            0.3145349396514471, 0.31448369068758164],
           [0.0, 0.0, 0.0, ..., 0.1533318528599792,
            0.15297283383566285, 0.15261489231834569],
           [--, --, --, ..., --, --, --]],

          [[0.0, 0.0, 0.0, ..., 0.31235523941899623,
            0.3123008275505995, 0.3122466849978085],
           [0.0, 0.0, 0.0, ..., 0.2105912409048617,
            0.21047195271051522, 0.21035282326223093],
           [0.0, 0.0, 0.0, ..., 0.23923576356367304,
            0.2394296315467117, 0.2396239399617004]],

          [[--, --, --, ..., --, --, --],
           [--, --, --, ..., --, --, --],
           [--, --, --, ..., --, --, --]],

          [[--, --, --

In [65]:
area_list = []
for subject_ind, subject in enumerate(subjects):

    for cpos_ind in range(len(func_entrop[0,0,:,0,0])):
        cpos = laterality[cpos_ind]
        x_temp = x_val[subject_ind, 0,0, ...].tolist()
        
        sample_integrals = func_entrop[:, subject_ind, cpos_ind, :, :]
        sample_integrals_mean = ma.mean(sample_integrals,axis=0)
        
        area = np.trapz(sample_integrals_mean, x_temp)
        area_list.append((subject, cpos, area))

filt_positions = list(set((a[1] for a in area_list)))

In [36]:
filt_positions

['L', 'LM2', 'LM', 'LM1', 'M', 'LL']

In [66]:
area_list


[('amap01',
  'LL',
  masked_array(data=[[71.30513267159995, 126.63029324895012, --]],
               mask=[[False, False,  True]],
         fill_value=1e+20)),
 ('amap01',
  'L',
  masked_array(data=[[59.05598514287023, 65.54924870073118, --]],
               mask=[[False, False,  True]],
         fill_value=1e+20)),
 ('amap01',
  'LM',
  masked_array(data=[[54.75603366346807, 52.52972657868992,
                      79.1375719972631]],
               mask=[[False, False, False]],
         fill_value=1e+20)),
 ('amap01',
  'LM1',
  masked_array(data=[[--, --, --]],
               mask=[[ True,  True,  True]],
         fill_value=1e+20,
              dtype=float64)),
 ('amap01',
  'LM2',
  masked_array(data=[[--, --, --]],
               mask=[[ True,  True,  True]],
         fill_value=1e+20,
              dtype=float64)),
 ('amap01',
  'M',
  masked_array(data=[[79.41600926558172, 51.09765540193598,
                      76.26177348951504]],
               mask=[[False, False, False]

In [67]:
pvalues = []
means = []
lab = []
labels = []

for pos_ind_one, p_one in enumerate(filt_positions):    
    for pos_ind_two, p_two in enumerate(filt_positions):         
        if pos_ind_one == pos_ind_two:
            pvalues.append((0))
            lab.append(p_one)
            means.append(0)
            labels.append(p_one)
            continue
        group_one = np.array(list(a[2] for a in area_list if a[1] == p_one))
        group_two = np.array(list(a[2] for a in area_list if a[1] == p_two))
        ttest = stats.ttest_rel(group_one, group_two, alternative='two-sided')
        pvalues.append(ttest.pvalue)
        ci = ttest.confidence_interval(.95)
        # pvalues.append((p_one, p_two, f"{stats.ttest_rel(group_one, group_two, alternative='two-sided')[1]}"))
        t = group_one - group_two
        means.append(t.mean())
        lab.append(t.mean())#f"{t.mean():.3f}\n({ci.low:.2f}, {ci.high:.2f})\n{ttest.pvalue:.3f}")

pvalues = np.array(pvalues)
pvalues = pvalues.reshape(4,4)

means = np.array(means)
means = means.reshape(4,4)

lab = np.array(lab)
lab = lab.reshape(4,4)
# unique_pvalues = set(tuple(sorted(t)) for t in pvalues)

mask = np.tril(np.ones_like(means), k=-1)
# mask = np.tril(np.ones_like(mask), k=-1)
plt.figure(figsize=(15, 5))
ax =sns.heatmap(means, xticklabels= labels, annot = lab, yticklabels= labels, mask = mask, fmt = "", cbar = False,center = 0, cmap = "coolwarm")

hatch_mask = pvalues > 0.1
handles = []

for i in range (pvalues.shape[0]):
  for j in range(pvalues.shape[1]):
      if hatch_mask[i, j]:
        ax.add_patch(plt.Rectangle((j, i), 1, 1, fill=False, hatch='////', edgecolor='white'))
handles.append(plt.Rectangle((0, 0), 0, 0, color='white', ec='black',
                    hatch='', label="Significant"))
handles.append(plt.Rectangle((0, 0), 0, 0, color='white', ec='black',
                    hatch='////', label="Not Significant"))
handles.append(plt.Rectangle((0, 0), 0, 0, color='blue', ec='black',
                    hatch='', label="Row Element Lower Threshold"))
handles.append(plt.Rectangle((0, 0), 0, 0, color='Red', ec='black',
                    hatch='', label="Row Element Higher Threshold"))
ax.legend(handles=handles, loc='lower left', bbox_to_anchor=(0, .1),
          handlelength=2, handleheight=2, frameon=False)
ax.tick_params(left= False,labelleft = False, bottom= False,labelbottom= False,right=True, top=True, labelright=True, labeltop=True, labelrotation=0)

ax.yaxis.tick_right()
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (36,) + inhomogeneous part.